# Model Distributions

Using the static agent model, comparing the distributions derived from different decisionmaking models.

In [1]:
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


from small_crisis import CrisisModel, CrisisAgent, CrisisWorld

from consul.agent_types.rl_agent import RLAgent
from consul.agent_types.cbr_agent import CaseBased_RLAgent


## Setting up agent and model types

In [2]:
class LearningAgent(RLAgent, CrisisAgent):
    pass

In [3]:
class CBRLPlayer(CaseBased_RLAgent, CrisisAgent):
    pass

In [4]:
class CBCrisisModel(CrisisModel):
    def __init__(self, agents):
        agent1, agent2 = agents
        if hasattr(agent1, "set_new_case"):
            agent1.set_new_case(self.make_coordinates(agent2))
            agent2.set_new_case(self.make_coordinates(agent1))
        super().__init__(agents)
    
    @staticmethod
    def make_coordinates(agent):
        '''
        Extract a coordinate tuple from a crisis agent
        '''
        return (agent.mil_strength, agent.assets, agent.bloc * 100)

In [5]:
class ChangingWorld(CrisisWorld):
    def step(self):
        
        for agent in self.agents:
            agent.mil_strength += random.normalvariate(0, 5)
            agent.mil_strength = max(1, min(agent.mil_strength, 100))
            agent.assets += random.normalvariate(0, 5)
            agent.assets = max(1, min(agent.assets, 100))
            if random.random() < 0.05:
                agent.bloc = 1 - agent.bloc
        super().step()

## Experiment 2.1 -- 10 agents, $\alpha=0.1$

In [ ]:
agent_args = {"learning_rate": 0.1,
              "discount_factor": 0.9}
N = 10

## Experiment 2.2 -- 20 agents, $\alpha=0.1$

In [ ]:
agent_args = {"learning_rate": 0.1,
              "discount_factor": 0.9}
N = 20

In [ ]:
rl_models = []
cbrl_models = []

for i in range(100):
    seed = random.randint(1000, 100000)
    model_1 = ChangingWorld(LearningAgent, N, agent_args=agent_args, seed=seed)
    model_2 = ChangingWorld(CBRLPlayer, N, agent_args=agent_args, seed=seed)
    model_2.model_class = CBCrisisModel
    
    model_1.run(100)
    model_2.run(100)
    rl_models.append(model_1)
    cbrl_models.append(model_2)
    
    if i % 10 == 0:
        print(i)

In [ ]:
rl_tables = []
cbrl_tables = []

for i in range(100):
    m1 = rl_models[i]
    df1 = m1.dc.get_table_dataframe("Interactions")
    df1["Seed"] = m1.seed
    rl_tables.append(df1)
    
    m2 = cbrl_models[i]
    df2 = m2.dc.get_table_dataframe("Interactions")
    df2["Seed"] = m2.seed
    cbrl_tables.append(df2)

In [ ]:
rl_table = pd.concat(rl_tables)
cbrl_table = pd.concat(cbrl_tables)

In [ ]:
with open("data_out/ex_2.1.pickle", "wb") as f:
    pickle.dump((rl_table, cbrl_table), f)

## Experiment 2.3 - Vary learning rates

In [ ]:
agent_args = {"learning_rate": 0.1,
              "discount_factor": 0.9}
N = 10
num_iter = 100

In [ ]:
rl_models = []
cbrl_models = []

for learning_rate in np.linspace(0.1, 1, 10):
    agent_args = {"learning_rate": learning_rate,
              "discount_factor": 1-learning_rate}
    
    for i in range(num_iter):
        seed = random.randint(1000, 100000)
        model_1 = ChangingWorld(LearningAgent, N, agent_args=agent_args, seed=seed)
        model_2 = ChangingWorld(CBRLPlayer, N, agent_args=agent_args, seed=seed)
        model_2.model_class = CBCrisisModel

        model_1.run(100)
        model_2.run(100)
        rl_models.append(model_1)
        cbrl_models.append(model_2)
    
    print(learning_rate)
    

In [ ]:
rl_tables = []
cbrl_tables = []
run_table = []

for i in range(len(rl_models)):
    m1 = rl_models[i]
    df1 = m1.dc.get_table_dataframe("Interactions")
    df1["Seed"] = m1.seed
    rl_tables.append(df1)
    
    m2 = cbrl_models[i]
    df2 = m2.dc.get_table_dataframe("Interactions")
    df2["Seed"] = m2.seed
    cbrl_tables.append(df2)
    
    run_row = m1.agent_args.copy()
    run_row["Seed"] = m1.seed
    run_table.append(run_row)

In [ ]:
rl_table = pd.concat(rl_tables)
cbrl_table = pd.concat(cbrl_tables)
seed_table = pd.DataFrame(run_table)
with open("data_out/ex_2.3.pickle", "wb") as f:
    pickle.dump((rl_table, cbrl_table, seed_table), f)

In [ ]:
# Sidebar: how likely was I to get a seed collission?
p_nocol = 1
for i in range(499):
    p_nocol *= (999000-i)/999000
print(1 - p_nocol)


# Experiment 2.4 - Delta-based coordinates

In [6]:
class CBCrisisModel2(CrisisModel):
    def __init__(self, agents):
        agent1, agent2 = agents
        if hasattr(agent1, "set_new_case"):
            agent1.set_new_case(self.make_coordinates(agent1, agent2))
            agent2.set_new_case(self.make_coordinates(agent2, agent1))
        super().__init__(agents)
    
    @staticmethod
    def make_coordinates(agent, alter):
        '''
        Extract a coordinate tuple from a crisis agent
        '''
        d_mil = agent.mil_strength - alter.mil_strength
        d_assets = agent.assets - alter.assets
        d_bloc = (agent.bloc == alter.bloc) * 100
        return (d_mil, d_assets, d_bloc)

In [8]:
agent_args = {"learning_rate": 0.5,
              "discount_factor": 0.5}
N = 10

In [ ]:
rl_models = []
cbrl_models = []

for i in range(100):
    seed = random.randint(1000, 100000)
    model_1 = ChangingWorld(LearningAgent, N, agent_args=agent_args, seed=seed)
    model_2 = ChangingWorld(CBRLPlayer, N, agent_args=agent_args, seed=seed)
    model_2.model_class = CBCrisisModel2
    
    model_1.run(100)
    model_2.run(100)
    rl_models.append(model_1)
    cbrl_models.append(model_2)
    
    if i % 10 == 0:
        print(i)

In [ ]:
rl_tables = []
cbrl_tables = []

for i in range(100):
    m1 = rl_models[i]
    df1 = m1.dc.get_table_dataframe("Interactions")
    df1["Seed"] = m1.seed
    rl_tables.append(df1)
    
    m2 = cbrl_models[i]
    df2 = m2.dc.get_table_dataframe("Interactions")
    df2["Seed"] = m2.seed
    cbrl_tables.append(df2)

In [ ]:
rl_table = pd.concat(rl_tables)
cbrl_table = pd.concat(cbrl_tables)
with open("data_out/ex_2.4.pickle", "wb") as f:
    pickle.dump((rl_table, cbrl_table), f)

# Experiment 2.5 - $\Delta$-coords, vary learning rates

In [9]:
rl_models = []
cbrl_models = []

all_seeds = set()
num_iter = 100

for learning_rate in np.linspace(0.1, 1, 10):
    agent_args = {"learning_rate": learning_rate,
              "discount_factor": 1-learning_rate}
    
    for i in range(num_iter):
        seed = random.randint(1000, 100000)
        while seed in all_seeds:
            seed = random.randint(1000, 100000)
        all_seeds.add(seed)
        
        model_1 = ChangingWorld(LearningAgent, N, agent_args=agent_args, seed=seed)
        model_2 = ChangingWorld(CBRLPlayer, N, agent_args=agent_args, seed=seed)
        model_2.model_class = CBCrisisModel2

        model_1.run(100)
        model_2.run(100)
        rl_models.append(model_1)
        cbrl_models.append(model_2)
    
    print(learning_rate)

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


In [10]:
rl_tables = []
cbrl_tables = []
run_table = []

for i in range(len(rl_models)):
    m1 = rl_models[i]
    df1 = m1.dc.get_table_dataframe("Interactions")
    df1["Seed"] = m1.seed
    rl_tables.append(df1)
    
    m2 = cbrl_models[i]
    df2 = m2.dc.get_table_dataframe("Interactions")
    df2["Seed"] = m2.seed
    cbrl_tables.append(df2)
    
    run_row = m1.agent_args.copy()
    run_row["Seed"] = m1.seed
    run_table.append(run_row)

rl_table = pd.concat(rl_tables)
cbrl_table = pd.concat(cbrl_tables)
seed_table = pd.DataFrame(run_table)
with open("data_out/ex_2.5.pickle", "wb") as f:
    pickle.dump((rl_table, cbrl_table, seed_table), f)